### Prueba de entrada

Presenta la ruta del repositorio donde se encuentra un cuaderno  de jupyter notebook con todas tus respuestas.

#### **Pregunta 1**

Estás organizando una fiesta. En preparación, vas a preparar una bebida mezclando tres tipos diferentes de jugo de fruta: **manzana**, **plátano** y **zanahoria**. Llamemos a los jugos **A**, **B** y **C**.

Quieres decidir qué fracción de la bebida debe estar compuesta por cada tipo de jugo, de tal manera que el **número máximo posible de personas** que asistan a la fiesta **la disfruten**.

Cada persona tiene una fracción mínima de cada uno de los 3 jugos que desea tener en la bebida. Solo les gustará la bebida si la fracción de cada uno de los 3 jugos en la mezcla es **mayor o igual** a su fracción mínima para ese jugo.

Tu objetivo es **determinar el número máximo de personas** que puedes satisfacer.

##### **Entrada**

- Una línea con un entero `T`, que representa el número de casos de prueba.

Para cada caso de prueba, habrá:

- Una línea con un entero `N`, el número de personas que van a la fiesta.
- `N` líneas adicionales, cada una para una persona, conteniendo tres números enteros separados por espacios: `"A B C"`, que representan la fracción mínima de cada jugo que desea esa persona en la bebida.

Los valores de `A`, `B` y `C` están entre **0 y 10,000** inclusive, y representan fracciones en partes por diez mil.  
Se garantiza que para cada persona se cumple: `A + B + C < 10,000`.

También se garantiza:
```
1 ≤ T ≤ 2
1 ≤ N ≤ 5000
```
**Salida**

- Se deben imprimir `T` líneas, una por cada caso de prueba, en el orden en que aparecen en la entrada.
- Cada línea debe contener la cadena:

```
Caso #X: Y
```

Donde:
- `X` es el número del caso de prueba (empezando en 1),
- `Y` es el número máximo de personas que disfrutarán la bebida preparada.

**Ejemplo de entrada 1**

```
2
3
10000 0 0
0 10000 0
0 0 10000
3
5000 0 0
0 2000 0
0 0 4000
```

**Ejemplo de salida 1**

```
Caso #1: 1
Caso #2: 2
```
**Ejemplo de entrada 2**

```
1
5
0 1250 0
3000 0 3000
1000 1000 1000
2000 1000 2000
1000 3000 2000
```

**Ejemplo de salida 2**

```
Caso #1: 5
```


In [195]:
from typing import List
from copy import deepcopy

In [196]:
MAX_TOTAL = 10000  # Límite máximo de la suma de jugos permitida por caso

In [197]:
def parse_input(input_str: str) -> List[List[List[int]]]:
    """
    Parsea una cadena de entrada con múltiples casos de prueba.

    Cada caso comienza con una línea que indica la cantidad de personas (columnas),
    seguida de N líneas donde cada línea representa los valores de jugos por persona (filas).

    Retorna una lista de matrices: cada matriz representa un caso de prueba con formato [jugos][personas].
    """
    # Eliminar líneas vacías y limpiar espacios
    lines = list(filter(None, map(str.strip, input_str.strip().split('\n'))))

    # Obtener la cantidad total de casos
    num_cases = int(lines.pop(0))
    cases = []
    idx = 0

    # Procesar cada caso
    while idx < len(lines):
        num_people = int(lines[idx])  # Número de personas (columnas de la matriz)
        idx += 1
        case_matrix = []

        # Leer N filas correspondientes a las personas
        for _ in range(num_people):
            row = list(map(int, lines[idx].split()))  # Convertir cada línea en lista de enteros
            case_matrix.append(row)
            idx += 1

        # Transponer la matriz: de [personas][jugos] a [jugos][personas]
        transposed = list(map(list, zip(*case_matrix)))
        cases.append(transposed)

    return cases

def get_max_per_juice(matrix: List[List[int]]) -> List[int]:
    # Obtiene el valor máximo de cada tipo de jugo entre todas las personas
    return [max(row) for row in matrix]

def get_total_max(matrix: List[List[int]]) -> int:
    # Suma los valores máximos por tipo de jugo para obtener el total
    return sum(get_max_per_juice(matrix))

def is_valid(matrix: List[List[int]]) -> bool:
    # Verifica si la suma total de jugos no excede el límite permitido
    return get_total_max(matrix) <= MAX_TOTAL

def find_person_with_highest_demand(matrix: List[List[int]]) -> int:
    """
    Encuentra el índice de la persona (columna) que tiene la mayor demanda de jugo
    en toda la matriz, sin importar el tipo de jugo.
    """
    max_value = -1
    person_index = -1

    # Recorrer toda la matriz buscando el valor más alto y su índice de columna
    for row in matrix:
        for idx, value in enumerate(row):
            if value > max_value:
                max_value = value
                person_index = idx
    return person_index

def exclude_person(matrix: List[List[int]], person_idx: int) -> List[List[int]]:
    """
    Elimina a la persona indicada (columna) de la matriz.
    Devuelve una nueva matriz sin esa persona.
    """
    return [[val for i, val in enumerate(row) if i != person_idx] for row in matrix]

def solve_iterative(matrix: List[List[int]]) -> int:
    """
    Algoritmo principal que determina cuántas personas pueden ser atendidas
    sin exceder el límite de jugo total. Usa enfoque iterativo.

    Elimina a la persona con mayor demanda hasta cumplir con la restricción.
    """
    matrix = deepcopy(matrix)  # Copia la matriz para no modificar la original

    while True:
        # Si no quedan personas, se retorna 0
        if not matrix or not matrix[0]:
            return 0

        # Obtener los valores máximos por jugo
        max_values = get_max_per_juice(matrix)
        total = sum(max_values)

        # Si la suma está dentro del límite, se puede atender a todos los actuales
        if total <= MAX_TOTAL:
            return len(matrix[0])

        # Verificar si falta exactamente el último jugo para completar el total
        remaining = MAX_TOTAL - sum(max_values[:-1])
        if remaining == max_values[-1]:
            return len(matrix[0])

        # Si no cumple, eliminar a la persona con mayor impacto
        person_to_remove = find_person_with_highest_demand(matrix)
        matrix = exclude_person(matrix, person_to_remove)

def process_all_cases(test_cases: List[List[List[int]]]) -> List[int]:
    """
    Procesa una lista de casos de prueba y muestra cuántas personas pueden ser atendidas en cada uno.

    test_cases: lista de matrices [jugos][personas]
    """
    results: List[int] = []
    for idx, case in enumerate(test_cases):
        result = solve_iterative(case)
        results.append(result)
        print(f"Caso #{idx + 1}: {result}")

In [198]:
raw_input = '''
2
3
10000 0 0
0 10000 0
0 0 10000
3
5000 0 0
0 2000 0
0 0 4000
'''

test_cases = parse_input(raw_input)
process_all_cases(test_cases)

Caso #1: 1
Caso #2: 2


In [199]:
raw_input = '''
1
5
0 1250 0
3000 0 3000
1000 1000 1000
2000 1000 2000
1000 3000 2000
'''

test_cases = parse_input(raw_input)
process_all_cases(test_cases)

Caso #1: 5


#### **Pregunta 2**

Durante el verano, las ciudades antiguas de Europa se llenan de turistas que recorren las calles y visitan los puntos de interés.

Muchas de estas ciudades fueron construidas de manera orgánica, sin seguir un plan arquitectónico definido, pero curiosamente, su crecimiento sigue un patrón común: las ciudades comienzan con **tres puntos de interés**, en los cuales **cada par está conectado** por una calle bidireccional. Luego, de manera gradual, se agregan nuevos puntos de interés.  
Cada nuevo punto se conecta mediante **dos calles bidireccionales** a **dos puntos anteriores diferentes**, que **ya estaban conectados directamente entre sí**.

Un turista desea hacer un recorrido visitando **la mayor cantidad posible de puntos de interés diferentes**. El recorrido puede comenzar en **cualquier punto**, pero debe terminar en el **mismo punto**.

Las reglas del recorrido son:

- Cada **calle** puede recorrerse **a lo sumo una vez**.
- Cada **punto de interés** puede visitarse **a lo sumo una vez**, excepto por el **punto de inicio**, que se visitará exactamente dos veces (al inicio y al final).


**Objetivo**

Dada la descripción de cómo creció la ciudad, determina **el mayor número de puntos de interés diferentes** que se pueden visitar en un solo recorrido.


**Entrada**

- La primera línea contiene un número entero `T`, que representa el número de casos de prueba.

Para cada caso de prueba:

- Una línea con un entero `N`,  el número total de puntos de interés en la ciudad.  
  Los puntos se numeran del `1` al `N`.

  - Los puntos `1`, `2` y `3` son los originales con los que la ciudad empezó.
  - Los puntos `4` hasta `N` son los que se añadieron posteriormente.

- Las siguientes `N -3` líneas contienen **dos enteros separados por espacio**, `A B`, que indican que el punto correspondiente se conecta mediante calles a los puntos `A` y `B`.

  - La primera de estas líneas corresponde al punto `4`, la segunda al `5`, y así sucesivamente.

**Salida**

Para cada caso de prueba, se debe imprimir una línea con el siguiente formato:

```
Caso #x: y
```

Donde:
- `x` es el número del caso de prueba (comenzando desde 1),
- `y` es el **mayor número de puntos de interés** que se pueden visitar en un recorrido válido por esa ciudad.

**Límites**

- `1 ≤ T ≤ 50`  
- `4 ≤ N ≤ 1000`

**Ejemplo de entrada**

```
2
5
1 2
2 1
6
1 2
1 4
4 5
```
**Ejemplo de salida**

```
Caso #1: 4
Caso #2: 6
```

In [200]:
from typing import List, Tuple

In [201]:
from typing import List, Tuple  # Importamos tipos para anotación de funciones

def parse_input(input_str: str) -> List[Tuple[int, List[Tuple[int, int]]]]:
    """
    Parsea una entrada en formato texto para múltiples casos de prueba.

    Formato esperado:
    - Primera línea: número de casos de prueba (T).
    - Luego, para cada caso:
        - Una línea con el número total de nodos N.
        - N - 3 líneas con pares de nodos que indican las conexiones.

    Retorna:
    - Lista de tuplas: cada tupla representa un caso (n, conexiones).
    """
    # Separamos el texto en líneas, quitando espacios en blanco y líneas vacías
    lines = list(filter(None, map(str.strip, input_str.strip().split('\n'))))

    # Leemos la cantidad de casos de prueba desde la primera línea
    num_cases = int(lines.pop(0))

    test_cases = []  # Aquí se almacenarán todos los casos parseados
    i = 0  # Índice para recorrer las líneas

    # Recorremos las líneas restantes para construir los casos
    while i < len(lines):
        n = int(lines[i])  # Número total de nodos para este caso
        i += 1

        connections = []  # Lista de pares de conexiones para este caso

        # Leemos N - 3 líneas con pares de nodos conectados
        for _ in range(n - 3):
            a, b = map(int, lines[i].split())  # Convertimos cada línea en un par de enteros
            connections.append((a, b))  # Agregamos la conexión a la lista
            i += 1

        # Agregamos el caso como una tupla (n, conexiones)
        test_cases.append((n, connections))

    return test_cases

def build_graph(n: int, connections: List[Tuple[int, int]]) -> List[List[int]]:
    """
    Construye el grafo en forma de lista de adyacencia.

    - Los nodos 1, 2 y 3 forman un triángulo base (todos conectados entre sí).
    - Cada nuevo nodo (desde el 4) se conecta a dos nodos anteriores.

    Retorna:
    - Una lista donde el índice representa un nodo y el contenido es su lista de vecinos.
    """
    # Creamos una lista vacía para cada nodo (índice 0 no se usa, por eso n + 1)
    graph = [[] for _ in range(n + 1)]

    # Creamos el triángulo base: nodos 1, 2 y 3 conectados entre sí
    graph[1].extend([2, 3])
    graph[2].extend([1, 3])
    graph[3].extend([1, 2])

    # Agregamos las conexiones de los nodos nuevos (4 en adelante)
    for i, (a, b) in enumerate(connections):
        new_node = i + 4  # Cada nuevo nodo se numera secuencialmente desde el 4
        graph[new_node].extend([a, b])  # Conectamos al nuevo nodo con los anteriores
        graph[a].append(new_node)  # Y también agregamos el nuevo nodo como vecino de los anteriores
        graph[b].append(new_node)

    return graph  # Devolvemos la lista de adyacencia construida

def find_longest_cycle(
    graph: List[List[int]],
    start: int,
    current: int,
    visited: List[bool],
    length: int
) -> int:
    """
    Usa búsqueda en profundidad (DFS) para encontrar el ciclo más largo
    que comienza y termina en el nodo `start`.

    Parámetros:
    - graph: grafo como lista de adyacencia
    - start: nodo de inicio y destino del ciclo
    - current: nodo actual en la exploración
    - visited: lista booleana que indica qué nodos han sido visitados
    - length: longitud actual del camino recorrido

    Retorna:
    - Longitud del ciclo más largo encontrado desde `start`
    """
    max_length = 0  # Variable para guardar la mayor longitud encontrada

    # Recorremos todos los vecinos del nodo actual
    for neighbor in graph[current]:
        if neighbor == start and length >= 3:
            # Si volvemos al nodo inicial y el ciclo tiene al menos 3 nodos, es válido
            max_length = max(max_length, length)
        elif not visited[neighbor]:
            # Si el vecino no ha sido visitado, lo marcamos y seguimos explorando
            visited[neighbor] = True
            # Llamada recursiva incrementando la longitud del camino
            max_length = max(
                max_length,
                find_longest_cycle(graph, start, neighbor, visited, length + 1)
            )
            visited[neighbor] = False  # Backtracking: desmarcamos el nodo para probar otros caminos

    return max_length  # Retornamos la mayor longitud de ciclo encontrada

def solve_case(n: int, connections: List[Tuple[int, int]]) -> int:
    """
    Resuelve un caso de prueba:
    - Construye el grafo con los nodos y conexiones.
    - Calcula el ciclo más largo posible.

    Retorna:
    - Longitud del ciclo más largo encontrado.
    """
    graph = build_graph(n, connections)  # Construimos el grafo

    longest = 0  # Variable para guardar la longitud máxima entre todos los ciclos

    # Probaremos iniciar un ciclo desde cada nodo
    for start in range(1, n + 1):
        visited = [False] * (n + 1)  # Reiniciamos lista de nodos visitados
        visited[start] = True  # Marcamos el nodo de inicio
        # Ejecutamos DFS desde este nodo y actualizamos la longitud máxima si es necesario
        longest = max(longest, find_longest_cycle(graph, start, start, visited, 1))

    return longest  # Retornamos la longitud del ciclo más largo encontrado

def process_all_cases(cases: List[Tuple[int, List[Tuple[int, int]]]]) -> List[int]:
    """
    Procesa todos los casos de prueba, resolviendo cada uno y mostrando su resultado.

    Parámetro:
    - cases: lista de tuplas, cada una con número de nodos y conexiones

    Retorna:
    - Lista de resultados por caso (longitud del ciclo más largo)
    """
    results: List[int] = []  # Lista para guardar los resultados de cada caso

    for idx, (n, connections) in enumerate(cases):
        result = solve_case(n, connections)  # Resolvemos el caso actual
        results.append(result)  # Guardamos el resultado
        print(f"Caso #{idx + 1}: {result}")  # Imprimimos el resultado en formato requerido

In [202]:
raw_input = '''
2
5
1 2
2 1
6
1 2
1 4
4 5
'''

cases = parse_input(raw_input)
process_all_cases(cases)

Caso #1: 4
Caso #2: 6
